### dGen Post-Run Analysis (Baseline vs Policy)
 - Loads per-state CSVs (baseline/policy)
 - Computes portfolio & cumulative bill savings (cohort carry-forward via `new_adopters`)
 - Aggregates: batt_kwh_cum, system_kw_cum, number_of_adopters, medians
 - Faceted plots by state (Baseline vs Policy)
 - EIA price compare

In [ ]:
# Set path to access helpers
from pathlib import Path
import sys

PARENT = Path.cwd().parent     # helpers live here
if str(PARENT) not in sys.path:
    sys.path.insert(0, str(PARENT))

In [ ]:
# Imports
from analysis_functions import *
import importlib, analysis_functions as af  # for reloads after editing the file
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")

# --- Paths / settings ---
ROOT_DIR = "/Volumes/Seagate Portabl/permit_power/dgen_runs/per_state_outputs"
RUN_ID   = "run_all_states_add_finance_series"
N_JOBS   = 8
cfg      = SavingsConfig(lifetime_years=30, cap_to_horizon=False)

# If you just edited analysis_functions.py, reload it:
importlib.reload(af)



#### Process all states (parallel)
##### Produces small, tidy DataFrames ready for plotting:
 - result["totals"] → batt_kwh_cum, system_kw_cum, number_of_adopters
 - result["portfolio_annual_savings"], result["cumulative_bill_savings"]
 - result["median_system_kw"], result["tech_2040"], result["lifetime_totals"], ...



In [ ]:
result   = process_all_states(root_dir=ROOT_DIR, run_id=RUN_ID, n_jobs=1, cfg=SavingsConfig)
peaks_df = process_all_states_peaks(root_dir=ROOT_DIR, run_id=RUN_ID, n_jobs=1)

facet_lines_national_totals(result, peaks_df=peaks_df)
# for k, df in result.items():
#     print(f"{k:28s} rows={len(df):,}")


In [ ]:
# Chloropleth map
df = choropleth_pv_delta_gw_policy_vs_baseline(
    outputs=result
    )

In [ ]:
# Grouped bar plot with cumulative adopters
us_bars = plot_us_cum_adopters_grouped(result)

In [ ]:
# Policy - Baseline lifetime savings totals
result['lifetime_totals'][result['lifetime_totals']['scenario'] == 'policy']['lifetime_savings_total'].sum()-result['lifetime_totals'][result['lifetime_totals']['scenario'] == 'baseline']['lifetime_savings_total'].sum()

In [ ]:
# Top 5 states for solar capacity
result['totals']['cum_new_system_kw'] = result['totals'].groupby(['state_abbr', 'scenario'], as_index = False)['new_system_kw'].cumsum()
result['totals']['initial_system_kw'] = result['totals']['system_kw_cum']-result['totals']['cum_new_system_kw']

capacity_2040 = (
    result['totals'][result['totals']['year'] == 2040][['state_abbr', 'scenario', 'system_kw_cum', 'cum_new_system_kw', 'initial_system_kw']]
    .pivot(index = ['state_abbr'], columns = ['scenario'], values = ['system_kw_cum', 'cum_new_system_kw', 'initial_system_kw'])
)

capacity_2040['diff'] = (capacity_2040['cum_new_system_kw']['policy']-capacity_2040['cum_new_system_kw']['baseline'])
capacity_2040['pct'] = (capacity_2040['diff']/capacity_2040['initial_system_kw']['baseline'])*100

#### Faceted choropleth of payback threshold under 5 yrs

In [ ]:
facet_choropleth_payback_continuous(
    root_dir=ROOT_DIR,
    shapefile_path="../../../data/states.shp",
    run_id=RUN_ID,
    year=2040
)


In [ ]:
payback_ts = build_payback_timeseries(
    root_dir=ROOT_DIR,
    run_id=RUN_ID,
    level = 'US'
)

payback_ts[payback_ts['year'] == 2040]

In [ ]:
eabs_state = build_eabs_calendar_timeseries(ROOT_DIR, run_id=RUN_ID, level="state")
eabs_us    = build_eabs_calendar_timeseries(ROOT_DIR, run_id=RUN_ID, level="US")

# Report national affordability in 2040
summarize_us_eabs_for_year(eabs_us, year=2040)

In [ ]:
# Top 5 states (policy, 2040) by EABS, with bills
table_top_states_by_eabs(eabs_state, year=2040, scenario="policy", top_n=5)

In [ ]:
# Build EABS (state-level)
eabs_state = build_eabs_calendar_timeseries(ROOT_DIR, run_id=RUN_ID, level="state")

# Build internal denominator from per-agent baseline outputs
denom = build_population_denominator_from_agents(ROOT_DIR, run_id=RUN_ID)

# US series
us_pct = compute_us_percent_savings_internal(eabs_state, denom)

# 2040 headline
us_2040 = us_pct[us_pct["year"] == 2040].set_index("scenario")
pol = us_2040.loc["policy"]
bas = us_2040.loc["baseline"]
print(
    f"In 2040, U.S. households avoid {pol['percent_savings']:.1%} of residential electricity spend "
    f"under the $1/W policy (vs {bas['percent_savings']:.1%} baseline), "
    f"equal to ${pol['total_savings_usd']/1e9:.1f}B in annual bill savings."
)

# Plot the US line
plot_us_percent_savings(us_pct)

# Top-5 states (2040, policy)
top5_2040 = table_top_states_by_percent_savings_internal(eabs_state, denom, year=2040, scenario="policy", top_n=5)
top5_2040


In [ ]:
result['totals']['cum_new_adopters'] = result['totals'].groupby(['state_abbr', 'scenario'], as_index = False)['new_adopters'].cumsum()
df_per_household = result['totals'][(result['totals']['year'] == 2040)].merge(result['lifetime_totals'], on = ['state_abbr', 'scenario'], how = 'left') 
df_per_household_agg = df_per_household.groupby(['scenario'], as_index = False).agg({'lifetime_savings_total':'sum', 'cum_new_adopters':'sum'})
df_per_household_agg['lifetime_savings_per_household'] = df_per_household_agg['lifetime_savings_total']/df_per_household_agg['cum_new_adopters']

In [ ]:
xlsx_path = export_compiled_results_to_excel(result, RUN_ID, peaks_df=peaks_df, include_national=True)
print("Wrote:", xlsx_path)